In [1]:
import re
import json
import ast

import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
df = pd.read_csv("../data/raw/all_curves.csv", engine='python')

In [3]:
df

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,y,project_names
0,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.859651,324.868269,349.87568,375.245424,39...","[-0.000148,-0.00016,-0.000173,-0.000182,-0.000...","[""ThermoelectricMaterials""]"
1,6006,10.1002/adma.201004200,Na0.0035Pb0.9965Se1,147,80,Temperature,Seebeck coefficient,K,V*K^(-1),"[317.791411,379.141104,384.04908,431.90184,452...","[0.000073,0.000074,0.000093,0.000102,0.000115,...","[""ThermoelectricMaterials"",""GeneralDB""]"
2,6444,10.1002/anie.200803934,Pb1.03Sb0.03Te1,434,81,Temperature,Seebeck coefficient,K,V*K^(-1),"[333.115876,340.549442,347.983009,355.416576,3...","[-0.000024,-0.000025,-0.000027,-0.000028,-0.00...","[""ThermoelectricMaterials""]"
3,5867,10.1016/j.actamat.2016.03.059,Pb0.76Mn0.24Te1Na0.04,248,83,Temperature,Seebeck coefficient,K,V*K^(-1),"[343.246753,376.363636,423.896104,472.207792,5...","[0.000188,0.000204,0.000234,0.000258,0.000279,...","[""ThermoelectricMaterials""]"
4,3485,10.1021/ja109138p,Pb1S0.84Te0.16Cl0.00134,389,84,Temperature,Seebeck coefficient,K,V*K^(-1),"[303.834784,320.966723,343.954054,368.609368,3...","[-0.000091,-0.000099,-0.000105,-0.000115,-0.00...","[""ThermoelectricMaterials""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
148915,1480,10.1016/s0925-8388(03)00411-0,CoSb2.94Se0.06,3507,2367,Temperature,Electrical resistivity,K,ohm*m,"[534.474,524.106,512.881,502.127,490.701,474.5...","[0.007838,0.008031,0.008129,0.008229,0.008381,...","[""ThermoelectricMaterials""]"
148916,1480,10.1016/s0925-8388(03)00411-0,CoSb2.92Se0.08,3508,2367,Temperature,Electrical resistivity,K,ohm*m,"[536.173,529.765,517.13,505.01,493.467,484.592...","[0.009183,0.01,0.010499,0.010628,0.01089,0.011...","[""ThermoelectricMaterials""]"
148917,1480,10.1016/s0925-8388(03)00411-0,CoSb2.76Se0.24,3509,2367,Temperature,Electrical resistivity,K,ohm*m,"[533.325,525.558,514.268,508.244,495.41,484.29...","[0.004756,0.004873,0.004933,0.005055,0.005179,...","[""ThermoelectricMaterials""]"
148918,1480,10.1016/s0925-8388(03)00411-0,CoSb2.68Se0.32,3510,2367,Temperature,Electrical resistivity,K,ohm*m,"[534.874,523.136,513.173,502.404,494.318,482.6...","[0.005243,0.005179,0.005243,0.005307,0.005307,...","[""ThermoelectricMaterials""]"


In [4]:
df['prop_y'].value_counts()

prop_y
Seebeck coefficient            33439
Thermal conductivity           24063
ZT                             19401
Electrical resistivity         18158
Power factor                   16058
Electrical conductivity        15876
Voltage                         6748
magnetization_per_weight        6150
Discharge capacity              5022
magnetization_per_volume        1220
Carrier mobility                1087
Magnetization                    889
Hall coefficient                 496
Magnetic field                   158
Magnetic field strength (H)      104
magnetization_Bohr                37
Temperature                       12
Cycle number                       2
Name: count, dtype: int64

In [5]:
df

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,y,project_names
0,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.859651,324.868269,349.87568,375.245424,39...","[-0.000148,-0.00016,-0.000173,-0.000182,-0.000...","[""ThermoelectricMaterials""]"
1,6006,10.1002/adma.201004200,Na0.0035Pb0.9965Se1,147,80,Temperature,Seebeck coefficient,K,V*K^(-1),"[317.791411,379.141104,384.04908,431.90184,452...","[0.000073,0.000074,0.000093,0.000102,0.000115,...","[""ThermoelectricMaterials"",""GeneralDB""]"
2,6444,10.1002/anie.200803934,Pb1.03Sb0.03Te1,434,81,Temperature,Seebeck coefficient,K,V*K^(-1),"[333.115876,340.549442,347.983009,355.416576,3...","[-0.000024,-0.000025,-0.000027,-0.000028,-0.00...","[""ThermoelectricMaterials""]"
3,5867,10.1016/j.actamat.2016.03.059,Pb0.76Mn0.24Te1Na0.04,248,83,Temperature,Seebeck coefficient,K,V*K^(-1),"[343.246753,376.363636,423.896104,472.207792,5...","[0.000188,0.000204,0.000234,0.000258,0.000279,...","[""ThermoelectricMaterials""]"
4,3485,10.1021/ja109138p,Pb1S0.84Te0.16Cl0.00134,389,84,Temperature,Seebeck coefficient,K,V*K^(-1),"[303.834784,320.966723,343.954054,368.609368,3...","[-0.000091,-0.000099,-0.000105,-0.000115,-0.00...","[""ThermoelectricMaterials""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
148915,1480,10.1016/s0925-8388(03)00411-0,CoSb2.94Se0.06,3507,2367,Temperature,Electrical resistivity,K,ohm*m,"[534.474,524.106,512.881,502.127,490.701,474.5...","[0.007838,0.008031,0.008129,0.008229,0.008381,...","[""ThermoelectricMaterials""]"
148916,1480,10.1016/s0925-8388(03)00411-0,CoSb2.92Se0.08,3508,2367,Temperature,Electrical resistivity,K,ohm*m,"[536.173,529.765,517.13,505.01,493.467,484.592...","[0.009183,0.01,0.010499,0.010628,0.01089,0.011...","[""ThermoelectricMaterials""]"
148917,1480,10.1016/s0925-8388(03)00411-0,CoSb2.76Se0.24,3509,2367,Temperature,Electrical resistivity,K,ohm*m,"[533.325,525.558,514.268,508.244,495.41,484.29...","[0.004756,0.004873,0.004933,0.005055,0.005179,...","[""ThermoelectricMaterials""]"
148918,1480,10.1016/s0925-8388(03)00411-0,CoSb2.68Se0.32,3510,2367,Temperature,Electrical resistivity,K,ohm*m,"[534.874,523.136,513.173,502.404,494.318,482.6...","[0.005243,0.005179,0.005243,0.005307,0.005307,...","[""ThermoelectricMaterials""]"


In [6]:
df[df['prop_y'] == 'Electrical resistivity']['DOI'].unique()

array(['10.1063/1.3603962', '10.1016/j.jallcom.2011.01.033',
       '10.1039/c5tc01781e', ..., '10.1016/j.proeng.2011.12.439',
       '10.1016/s0925-8388(03)00411-0', '10.1021/jacs.7b01851'],
      dtype=object)

In [7]:
def convert_string_to_list(string_value):    
    try:        
        return eval(string_value)
    except Exception as e:
        print(string_value)
        print(e)

df['y'] = df['y'].apply(convert_string_to_list)
df['x'] = df['x'].apply(convert_string_to_list)

In [8]:
df.dtypes

SID               int64
DOI              object
composition      object
sample_id         int64
figure_id         int64
prop_x           object
prop_y           object
unit_x           object
unit_y           object
x                object
y                object
project_names    object
dtype: object

In [9]:
df = df.explode(['x', 'y']).reset_index(drop=True)

In [10]:
df

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,y,project_names
0,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),299.859651,-0.000148,"[""ThermoelectricMaterials""]"
1,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),324.868269,-0.00016,"[""ThermoelectricMaterials""]"
2,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),349.87568,-0.000173,"[""ThermoelectricMaterials""]"
3,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),375.245424,-0.000182,"[""ThermoelectricMaterials""]"
4,6,10.1021/am405410e,Pb1.00025Zn0.02Te1.02I0.0005,113,79,Temperature,Seebeck coefficient,K,V*K^(-1),399.898544,-0.000193,"[""ThermoelectricMaterials""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2767825,41688,10.1021/jacs.7b01851,PbCoO3,46306,36159,Temperature,Electrical resistivity,K,ohm*m,254.600219,787.234097,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767826,41688,10.1021/jacs.7b01851,PbCoO3,46306,36159,Temperature,Electrical resistivity,K,ohm*m,308.731028,293.448888,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767827,41688,10.1021/jacs.7b01851,PbCoO3,46306,36159,Temperature,Electrical resistivity,K,ohm*m,377.335315,130.882963,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767828,41688,10.1021/jacs.7b01851,PbCoO3,46306,36159,Temperature,Electrical resistivity,K,ohm*m,426.701612,91.419525,"[""ThermoelectricMaterials"",""CondensedMatter""]"


In [11]:
for prop, count in df['prop_x'].value_counts(normalize=False).items():
    df.loc[df['prop_x'] == prop, prop] = pd.to_numeric(df['x'])

for prop, count in df['prop_y'].value_counts(normalize=False).items():
    df.loc[df['prop_y'] == prop, prop] = pd.to_numeric(df['y'])

In [12]:
df.dtypes

SID                             int64
DOI                            object
composition                    object
sample_id                       int64
figure_id                       int64
prop_x                         object
prop_y                         object
unit_x                         object
unit_y                         object
x                              object
y                              object
project_names                  object
Temperature                    object
Magnetic field strength (H)    object
Cycle number                   object
Discharge capacity             object
Charge capacity                object
Magnetic field                 object
Magnetization                  object
C rate                         object
Magnetic Field                 object
Voltage                        object
magnetization_per_weight       object
Electrical conductivity        object
magnetization_per_volume       object
Seebeck coefficient            object
Electrical r

In [13]:
df["figure_id"] = df["figure_id"].apply(str)
df["SID"] = df["SID"].apply(str)
df["sample_id"] = df["sample_id"].apply(str)

In [16]:
# frequent prop
df_fp = df.groupby(['prop_x', 'prop_y']).agg({'unit_x': 'first', 'unit_y': 'first'}).reset_index()

In [17]:
df_fp

,prop_x,prop_y,unit_x,unit_y
0,C rate,Cycle number,-,-
1,C rate,Discharge capacity,-,A*s/kg
2,C rate,Voltage,-,V
3,Carrier mobility,Seebeck coefficient,m^2*V^(-1)*s^(-1),V*K^(-1)
4,Charge capacity,Discharge capacity,A*s/kg,A*s/kg
5,Charge capacity,Voltage,A*s/kg,V
6,Cycle number,Discharge capacity,-,A*s/kg
7,Cycle number,Voltage,-,V
8,Discharge capacity,Voltage,A*s/kg,V
9,Electrical conductivity,Power factor,ohm^(-1)*m^(-1),W*m^(-1)*K^(-2)


In [35]:
from bokeh.plotting import figure, output_file, save, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool, Label, Div
from bs4 import BeautifulSoup

soup = BeautifulSoup("<html><head><title>Curve example</title></head><body><h1>Curve examples</h1></body></html>", "html.parser")
ul_tag = soup.new_tag("ul")

for index, row in df_fp.iterrows():
    prop_x = row['prop_x']
    prop_y = row['prop_y']
    unit_x = row['unit_x']
    unit_y = row['unit_y']
    label_x = f'{prop_x} ({unit_x})'
    label_y = f'{prop_y} ({unit_y})'

    selected_df = df[(df[prop_x].notnull()) & (df[prop_y].notnull())]
    
    source = ColumnDataSource(data=dict(
        x=selected_df[prop_x].values,
        y=selected_df[prop_y].values,
        SID=selected_df['SID'],
        figure_id=selected_df['figure_id'],
        sample_id=selected_df['sample_id'],
        composition=selected_df['composition']
    ))
    if prop_x == prop_y:
        continue
    if index == 3:
        break
    # continue

    p = figure(title=f'{prop_x} vs. {prop_y}', x_axis_label=label_x, y_axis_label=label_y, x_axis_type='log', y_axis_type='log')
    
    p.scatter('x', 'y', source=source, alpha=0.2)

    file_name = f'../reports/figures/curves/{prop_x}-{prop_y}'
    # ツールチップの設定
    hover = HoverTool(tooltips=[('SID', '@SID'), ('Figure ID', '@figure_id'), ('Sample ID', '@sample_id'), ('composition', '@composition')])
    p.add_tools(hover)
    
    # 説明文を作成
    # caption = Div(text="<h3>この線グラフはサンプルデータを示しています</h3>")

    # レイアウトを作成
    # layout = column(caption, p)

    output_file(filename=file_name + '.html')
    # export_png(p, filename=file_name + '.png')
    # show(p)   

    save(p)
    li_tag = soup.new_tag("li")
    a_tag = soup.new_tag("a")

    a_tag.string = f'{prop_x}-{prop_y}'    
    a_tag['href'] = f'./{prop_x}-{prop_y}.html'
    li_tag.append(a_tag)
    ul_tag.append(li_tag)

body_tag = soup.body
body_tag.append(ul_tag)

html_string = soup.prettify()
# HTML文字列をファイルに書き込み
with open('../reports/figures/curves/index.html', 'w') as file:
    file.write(html_string)



In [ ]:
!which python

python not found
